In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import os
import cv2

In [ ]:
def Image_Forgery_Predictor_Model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(2, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def recomprimir_imagen_tf(imagen, calidad=50):
    imagen_jpeg = tf.image.encode_jpeg(imagen, quality=calidad)
    imagen_recomprimida = tf.image.decode_jpeg(imagen_jpeg)
    return imagen_recomprimida

def preprocess_image(image):
    image_compressed = recomprimir_imagen_tf(image, calidad=50)
    diff = tf.abs(tf.cast(image, tf.int32) - tf.cast(image_compressed, tf.int32))
    resized = tf.image.resize(diff, (128, 128))
    return resized



In [ ]:
def train_model(images, labels, epochs=10):
    model = Image_Forgery_Predictor_Model()
    training_data = []
    for image in images:
        diff_image = preprocess_image(image)
        training_data.append(diff_image)
    training_data = np.array(training_data)
    labels = np.array(labels)
    
    # Dividir los datos en conjuntos de entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.2, random_state=42)
    
    # Convertir las etiquetas a una representación binaria
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
    
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
    return model


In [ ]:
def predict_image(model, input_image):
    input_diff_image = preprocess_image(input_image)
    input_diff_image = np.expand_dims(input_diff_image, axis=0)  # Añadir dimensión de lote
    predicted_label = model.predict(input_diff_image)
    if predicted_label[0][0] > predicted_label[0][1]:
        return "Tampered"
    else:
        return "Untampered"


In [ ]:
# Path to your CASIA2 dataset
PATH = '../data/CASIA2'

# Directories for authentic and tampered images
authentic_dir = os.path.join(PATH, 'Au')
tampered_dir = os.path.join(PATH, 'Tp2')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

def load_tif_image(file_path):
    # Cargar la imagen usando Pillow
    image = Image.open(file_path)
    # Convertir la imagen a un arreglo numpy
    image_array = np.array(image)
    # Convertir el arreglo numpy a un tensor de TensorFlow
    return tf.convert_to_tensor(image_array)

def load_images_from_directory(directory_path,n):
    images = []
    for filename in os.listdir(directory_path)[:n]:
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            image_path = os.path.join(directory_path, filename)
            image_tensor = load_tif_image(image_path)
            resized = tf.image.resize(image_tensor, IMG_SIZE)
            images.append(resized)
        elif filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(directory_path, filename)
            imagen_original = tf.io.read_file(image_path)
            imagen_original = tf.image.decode_jpeg(imagen_original, channels=3)
            images.append(imagen_original)
    return images

# Get file lists and labels
authentic_files = load_images_from_directory(authentic_dir, 100)
tampered_files = load_images_from_directory(tampered_dir, 100)
authentic_labels = [0] * len(authentic_files)
tampered_labels = [0] * len(tampered_files)

# Combine authentic and tampered data
all_files = authentic_files + tampered_files
all_labels = authentic_labels + tampered_labels

# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

In [ ]:
# Entrenar el modelo
model = train_model(train_files, train_labels, epochs=10)